# Semester Project: Exploring large language model performance in college-level project evaluation as a means to improving course quality

Skyler Brough and Trina Brough  
CS 533  
Fall 2025  

## Set up

In [14]:
from openpyxl import load_workbook
import json
import re

AGG_FUNCTIONS = {"SUM", "AVERAGE", "COUNT", "MAX", "MIN", "MEDIAN", "MODE"}

In [15]:

def extract_functions(formula):
    """Extracts function names from Excel formula."""
    if not formula or not isinstance(formula, str):
        return []
    return re.findall(r"\b[A-Z]+\b", formula)


In [11]:


CELL_REF_PATTERN = re.compile(r"([A-Z]+[0-9]+|\'[^\']+\'![A-Z]+[0-9]+)")

def extract_spreadsheet_metadata(file_path):
    """
    Extracts values, formulas, formatting, and explicit cell references from Excel.
    Returns a JSON string.
    """
    wb = load_workbook(file_path, data_only=False)
    all_sheets = {}

    for sheetname in wb.sheetnames:
        ws = wb[sheetname]
        sheet_info = {
            "sheet_name": sheetname,
            "max_row": ws.max_row,
            "max_column": ws.max_column,
            "has_charts": bool(ws._charts), 
            "chart_count": len(ws._charts),
            "cells": {}
        }

        for row in ws.iter_rows():
            for cell in row:
                coord = cell.coordinate
                formula = cell.value if cell.data_type == "f" else None
                functions = extract_functions(formula)
                aggregations = [f for f in functions if f.upper() in AGG_FUNCTIONS]

                # Extract all referenced cells in the formula
                references = []
                if formula:
                    references = CELL_REF_PATTERN.findall(formula)

                # Safely convert color objects to strings
                font_color = None
                if cell.font.color:
                    font_color = (
                        cell.font.color.rgb
                        if cell.font.color.type == "rgb"
                        else str(cell.font.color)
                    )

                fill_color = None
                if cell.fill.fgColor:
                    fill_color = (
                        cell.fill.fgColor.rgb
                        if cell.fill.fgColor.type == "rgb"
                        else str(cell.fill.fgColor)
                    )

                cell_info = {
                    "value": cell.value,
                    "formula": formula,
                    "functions": functions,
                    "aggregations": aggregations if aggregations else None,
                    "references": references if references else None,
                    "number_format": cell.number_format,
                    "font": {
                        "bold": cell.font.bold,
                        "italic": cell.font.italic,
                        "color": font_color,
                    },
                    "fill": {
                        "fgColor": fill_color,
                    },
                    "alignment": {
                        "horizontal": cell.alignment.horizontal,
                        "vertical": cell.alignment.vertical,
                    },
                    "comment": cell.comment.text if cell.comment else None,
                }

                sheet_info["cells"][coord] = cell_info

        all_sheets[sheetname] = sheet_info

    return json.dumps(all_sheets, indent=2)


In [7]:
#Saving individual elements without the entire metadata
from openpyxl import load_workbook

def load_workbook_all_sheets(path):
    """Loads the workbook and returns a dictionary of all sheets. Keys are sheet names, values are worksheet objects."""
    wb = load_workbook(path, data_only=True)
    sheets = {sheet.title: sheet for sheet in wb.worksheets}
    return sheets

def extract_data(sheet):
    """
    Extracts all rows from a worksheet as a list of lists. Skips rows that are completely empty."""
    rows = []

    # Iterate through all rows in the worksheet
    for row in sheet.iter_rows(values_only=True):
        row_values = []
        for cell in row:
            row_values.append(cell)
        # Check if the row has at least one non-empty cell
        has_data = False
        for value in row_values:
            if value is not None:
                has_data = True
                break
        # Only append rows that have data
        if has_data:
            rows.append(row_values)
    return rows


#Step 1: Load all sheets
sheets = load_workbook_all_sheets('./widrigellie.xlsx')  # replace with your file path

#Step 2: Loop through each sheet
for sheet_name, sheet in sheets.items():
    print(f"\n Sheet: {sheet_name}")
    
    #Step 3: Extract all rows
    all_rows = extract_data(sheet)
    
    #Step 4: Print a sample (first 5 rows) to test
    for row in all_rows[:5]:
        print(row)
    
    #Print total number of rows
    print(f"Total rows in sheet: {len(all_rows)}")




 Sheet: StarbucksDrinks
['Beverage_category', 'Beverage', 'Beverage_prep', 'Calories', ' Total Fat (g)', 'Trans Fat (g) ', 'Saturated Fat (g)', ' Sodium (mg)', ' Total Carbohydrates (g) ', 'Cholesterol (mg)', ' Dietary Fibre (g)', ' Sugars (g)', ' Protein (g) ', 'Vitamin A (% DV) ', 'Vitamin C (% DV)', ' Calcium (% DV) ', 'Iron (% DV) ', 'Caffeine (mg)']
['Coffee', 'Brewed Coffee', 'Short', 3, 0.1, 0, 0, 0, 5, 0, 0, 0, 0.3, 0, 0, 0, 0, 175]
['Coffee', 'Brewed Coffee', 'Tall', 4, 0.1, 0, 0, 0, 10, 0, 0, 0, 0.5, 0, 0, 0, 0, 260]
['Coffee', 'Brewed Coffee', 'Grande', 5, 0.1, 0, 0, 0, 10, 0, 0, 0, 1, 0, 0, 0, 0, 330]
['Coffee', 'Brewed Coffee', 'Venti', 5, 0.1, 0, 0, 0, 10, 0, 0, 0, 1, 0, 0, 0.02, 0, 410]
Total rows in sheet: 246

 Sheet: SaturatedFatsPivotTable
['Row Labels', 'Sum of Saturated Fat (g)']
['Classic Espresso Drinks', 2.7]
['CaffÃ¨ Americano', 0]
['CaffÃ¨ Latte', 0.8]
['CaffÃ¨ Mocha (Without Whipped Cream)', 0.6000000000000001]
Total rows in sheet: 47

 Sheet: ProteinbyMilk


In [ ]:
def save_metadata_to_json(input_xlsx, output_json):
    metadata_json = extract_spreadsheet_metadata(input_xlsx)
    with open(output_json, "w", encoding="utf-8") as f:
        f.write(metadata_json)
    print(f"Metadata saved to {output_json}")



if __name__ == "__main__":
    # Example usage
    input_file = "student_project.xlsx"    # path to spreadsheet
    output_file = "student_project_metadata.json"
    save_metadata_to_json(input_file, output_file)

In [12]:
test = extract_spreadsheet_metadata('./widrigellie.xlsx')

In [1]:
#test

## Data

In [ ]:
#Open AI API syntax
client.files.create(
    file=open("student_project_metadata.json", "rb"),
    purpose="assistants"
)


## Prompt Bank

In [ ]:
RUBRIC = [
    '30 unique data points',
    'aggregation',
    'graph',
    'something that scares you',
    'explanatory paragraph'
]



#Ideas:
You are a grading assistant evaluating spreadsheet projects.
The rubric contains 4 criteria: formula accuracy, cell references, formatting, and sheet organization.

The student’s spreadsheet metadata is provided.

Task: For each rubric item, give a score 0 or 1 and a short justification.


Ideas:
General
Split each rubric item apart
Provide one shot example

### Prompt 1: Simple structured grading

In [ ]:
prompt1 = '''
    Task:
        - Review the JSON representing a student spreadsheet.
        - Assign 1 if each rubric item is met, 0 if not.
        - Provide short comment for each.
        - Sum total_score.
        
    Return JSON:
        {
          "30 unique data points": {"score": X, "comment": "..."},
          "aggregation": {"score": X, "comment": "..."},
          "graph": {"score": X, "comment": "..."},
          "something that scares you": {"score": X, "comment": "..."},
          "explanatory paragraph": {"score": X, "comment": "..."},
          "total_score": XX
        }
        
    RUBRIC:
        
    
    JSON:
        {paste JSON here}

'''

### Prompt 2: Minimal output

In [ ]:
prompt2 = '''
    Task:
        - Assign 0 or 1 for each rubric item.
        - Include total_score only, no comments.
        
        Return JSON:
        {
          "30 unique data points": X,
          "aggregation": X,
          "graph": X,
          "something that scares you": X,
          "explanatory paragraph": X,
          "total_score": XX
        }

        RUBRIC:
        
        JSON:
        {paste JSON here}

'''

### Prompt 3: Yes/No style

In [ ]:
prompt3 = '''
    Task:
        - For each rubric item, write "Yes" if met, "No" if not.
        - Include optional short comment.
        
    Return JSON:
        {
          "30 unique data points": {"met": "Yes", "comment": "..."},
          "aggregation": {"met": "No", "comment": "..."},
          "graph": {"met": "Yes", "comment": "..."},
          "something that scares you": {"met": "Yes", "comment": "..."},
          "explanatory paragraph": {"met": "No", "comment": "..."},
          "total_score": XX
        }
        
    RUBRIC:
    
    JSON:
        {paste JSON here}

'''

### Prompt 4: Full reasoning per criterion

In [ ]:
prompt4 = '''
    Task:
        - Explain 1–2 sentences per rubric criterion.
        - Assign 1 if met, 0 if not.
        - Sum total_score.
        
    Return JSON:
        {
          "30 unique data points": {"score": X, "explanation": "..."},
          "aggregation": {"score": X, "explanation": "..."},
          "graph": {"score": X, "explanation": "..."},
          "something that scares you": {"score": X, "explanation": "..."},
          "explanatory paragraph": {"score": X, "explanation": "..."},
          "total_score": XX
        }
        
    RUBRIC:
    
    JSON:
        {paste JSON here}

'''

In [ ]:
### Prompt 5: Checklist with suggestions

In [ ]:
prompt5 = '''
   Task:
        - For each rubric item, assign 0 or 1.
        - Include one improvement suggestion if 0.
        
        Return JSON:
        {
          "30 unique data points": {"score": X, "suggestion": "..."},
          "aggregation": {"score": X, "suggestion": "..."},
          "graph": {"score": X, "suggestion": "..."},
          "something that scares you": {"score": X, "suggestion": "..."},
          "explanatory paragraph": {"score": X, "suggestion": "..."},
          "total_score": XX
        }

    RUBRIC:
        
    JSON:
        {paste JSON here}
 

'''

In [ ]:
### Prompt 6: Focus on aggregations

In [ ]:
prompt6 = '''
   Task:
    - Check for aggregation formulas: SUM, AVERAGE, COUNT, MAX, MIN, MEDIAN.
    - Score "aggregation" rubric item 1 if present, 0 if not.
    - Comment on formula usage.
    
    Return JSON:
    {
      "aggregation": {"score": X, "comment": "..."}
    }
    
    JSON:
    {paste JSON here}
     

'''

In [ ]:
### Prompt 7: Focus on scary functions

In [ ]:
prompt7 = '''
  Task:
    - Check for IF, XLOOKUP, VLOOKUP, INDEX/MATCH, Pivot Table, Conditional Formatting.
    - Score "something that scares you" 1 if any present, 0 if not.
    - Comment on which feature was used.
    
    Return JSON:
    {
      "something that scares you": {"score": X, "comment": "..."}
    }
    
    JSON:
    {paste JSON here}
     

'''

In [ ]:
### Prompt 8: Focus on graph/chart detection

In [ ]:
prompt8 = '''
  Task:
    - Detect any chart in the workbook.
    - Score "graph" rubric item 1 if present, 0 if not.
    - Comment on type of chart.
    
    Return JSON:
    {
      "graph": {"score": X, "comment": "..."}
    }
    
    JSON:
    {paste JSON here}
'''

In [ ]:
### Prompt 9: Data points detection

In [ ]:
prompt9 = '''
  Task:
    - Check if spreadsheet has at least 30 unique data points.
    - Score rubric item accordingly.
    - Comment on total unique entries.
    
    Return JSON:
    {
      "30 unique data points": {"score": X, "comment": "..."}
    }
    
    JSON:
    {paste JSON here}
'''

In [ ]:
### Prompt 10: Explanatory paragraph detection

In [ ]:
prompt10 = '''
  Task:
    - Detect text in a designated cell or comment for explanatory paragraph.
    - Score 1 if present, 0 if not.
    - Comment on location/length.
    
    Return JSON:
    {
      "explanatory paragraph": {"score": X, "comment": "..."}
    }
    
    JSON:
    {paste JSON here}
'''

In [ ]:
### Prompt 11: Aggregate grading

In [ ]:
prompt11 = '''
  Task:
    - Check all rubric items in one pass.
    - Assign 1/0 per item.
    - Include total_score.
    - Include brief comment per item.
    
    Return JSON:
    {
      "30 unique data points": {"score": X, "comment": "..."},
      "aggregation": {"score": X, "comment": "..."},
      "graph": {"score": X, "comment": "..."},
      "something that scares you": {"score": X, "comment": "..."},
      "explanatory paragraph": {"score": X, "comment": "..."},
      "total_score": XX
    }

    RUBRIC:
    
    JSON:
    {paste JSON here}
'''

In [ ]:
### Prompt 12: Minimal total + comment

In [ ]:
prompt12 = '''
  Task:
    - Only return total_score.
    - Optionally include one-line summary comment.
    
    Return JSON:
    {
      "total_score": XX,
      "comment": "Student included aggregation and chart but no paragraph."
    }

    RUBRIC:
    
    JSON:
    {paste JSON here}
'''

In [ ]:
### Prompt 13: Step-by-step reasoning

In [ ]:
prompt13 = '''
  Task:
    - For each rubric item, explain reasoning step-by-step:
      1. What was found in JSON.
      2. Whether it meets rubric criterion.
      3. Assign score 1 or 0.
    - Sum total_score.
    
    Return JSON:
    {
      "30 unique data points": {"score": X, "explanation": "..."},
      "aggregation": {"score": X, "explanation": "..."},
      "graph": {"score": X, "explanation": "..."},
      "something that scares you": {"score": X, "explanation": "..."},
      "explanatory paragraph": {"score": X, "explanation": "..."},
      "total_score": XX
    }
    
    JSON:
    {paste JSON here}
'''

In [ ]:
### Prompt 14: Short human-readable feedback

In [ ]:
prompt14 = '''
  Task:
    - Provide 3–5 sentence summary of spreadsheet.
    - Highlight rubric items met/missing.
    - Include numeric score.
    
    Return JSON:
    {
      "feedback": "...",
      "total_score": XX
    }
    
    JSON:
    {paste JSON here}
'''

In [ ]:
### Prompt 15: Multi-style output

In [ ]:
prompt15 = '''
  Task:
    - For each rubric item, provide:
      - score (1/0)
      - brief comment
      - improvement suggestion if not met
    - Include total_score
    - Provide optional overall feedback
    
    Return JSON:
    {
      "30 unique data points": {"score": X, "comment": "...", "suggestion": "..."},
      "aggregation": {"score": X, "comment": "...", "suggestion": "..."},
      "graph": {"score": X, "comment": "...", "suggestion": "..."},
      "something that scares you": {"score": X, "comment": "...", "suggestion": "..."},
      "explanatory paragraph": {"score": X, "comment": "...", "suggestion": "..."},
      "total_score": XX,
      "overall_feedback": "..."
    }
    
    JSON:
    {paste JSON here}
'''

### LLM Spreadsheet Grading Test Table

| Prompt # | Prompt Style | Expected Output Keys | Notes / Focus | Model 1 Score | Model 2 Score | Model 3 Score | Comments |
|----------|--------------|--------------------|---------------|---------------|---------------|---------------|---------|
| 1 | Simple structured grading | 30 unique data points, aggregation, graph, something that scares you, explanatory paragraph, total_score | Checks binary scores with short comments | | | | |
| 2 | Minimal output | 30 unique data points, aggregation, graph, something that scares you, explanatory paragraph, total_score | Only numeric scores, fast parsing | | | | |
| 3 | Yes/No style | 30 unique data points, aggregation, graph, something that scares you, explanatory paragraph, total_score | “Yes”/“No” instead of 1/0 | | | | |
| 4 | Full reasoning per criterion | 30 unique data points, aggregation, graph, something that scares you, explanatory paragraph, total_score | LLM explains each decision | | | | |
| 5 | Checklist with suggestions | 30 unique data points, aggregation, graph, something that scares you, explanatory paragraph, total_score | Suggestions included if 0 | | | | |
| 6 | Aggregation focus | aggregation | Checks SUM, AVERAGE, COUNT, MAX, MIN, MEDIAN | | | | |
| 7 | Scary functions focus | something that scares you | IF, XLOOKUP, VLOOKUP, INDEX/MATCH, Pivot Table, Conditional Formatting | | | | |
| 8 | Graph detection | graph | Detect charts and type | | | | |
| 9 | Data points detection | 30 unique data points | Checks unique value count | | | | |
| 10 | Explanatory paragraph detection | explanatory paragraph | Text in cell or comment | | | | |
| 11 | Aggregate grading | all rubric items + total_score | One-pass scoring with brief comments | | | | |
| 12 | Minimal total + comment | total_score, comment | Single numeric output | | | | |
| 13 | Step-by-step reasoning | all rubric items + total_score | LLM explains reasoning stepwise | | | | |
| 14 | Short human-readable feedback | feedback, total_score | Text summary suitable for student | | | | |
| 15 | Multi-style output | all rubric items + total_score + overall_feedback | Combines score, comment, suggestion | | | | |


## Models

Recommended LLMs for this setup
1. OpenAI GPT-4 / GPT-4-turbo

Why it works well: Excellent at reasoning over structured data, multi-step logic, and following complex instructions. Handles large prompts well (good for multi-sheet JSON + rubric).

Pros: High reliability, great zero/few-shot reasoning.

Cons: Cost, and context window limits if the JSON is very large.

Best use: Start here — can even prompt it to output structured JSON with grades and feedback.

2. Claude 3 / Claude 3.5

Why it works: Very strong at following complex instructions and safe reasoning. Can process structured data effectively.

Pros: Often more consistent on instruction-following than GPT in some tasks.

Cons: API access, slightly lower adoption in open-source tooling.

Best use: Great alternative to GPT-4 if you want instruction-following to be ultra-consistent.

3. LLaMA 3 (or LLaMA 3 derivatives like MPT, Gemma 8B/27B)

Why it works: Open-source, can run locally. Fine-tuneable for grading tasks.

Pros: You can tweak behavior, process local JSON without sending data to the cloud.

Cons: Reasoning over large JSON is harder than GPT/Claude out-of-the-box; might need prompting tricks or fine-tuning.

Best use: Useful if you want a self-hosted, privacy-preserving solution.

4. Specialized “Spreadsheet-aware” LLMs

SpreadsheetLLM / SheetGPT / Unstract (specialized tools)

Why: Some frameworks are trained to understand spreadsheet-like structures and formulas.

Pros: Potentially better at interpreting formulas, references, and aggregations.

Cons: Less general; might be overkill if JSON already has formulas parsed.

Best use: Consider if you want automated formula checking + reasoning combined.

⚡ Practical Recommendation

Start with GPT-4-turbo:

Give it the JSON + rubric.

Prompt it to return structured output: {score: x, comments: "...", rubric_breakdown: {...}}.

Works even for nested formulas and multi-sheet references.

Use Claude 3.5 as a comparison:

See if it produces more consistent rubric-following output.

Optionally explore LLaMA 3 / MPT / Gemma if you need a local/private solution:

Likely need careful prompt engineering or fine-tuning to handle structured JSON as reliably as GPT/Claude.